#### This was a testing feature extractor with a different prompt template, it did not work as intended.

In [1]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import (
    HumanMessage,
)
from typing import List, Optional
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.llms import HuggingFaceEndpoint
from dotenv import load_dotenv
import os
import json
from langchain_google_genai import ChatGoogleGenerativeAI
from json import JSONDecodeError

c:\Users\tomce\miniconda3\envs\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Initialize LangChain model
llm = ChatGoogleGenerativeAI(model="gemini-pro", token=GOOGLE_API_KEY)

In [11]:
text="""
{
    "url": "https://www.neptun.mk/categories/mobilni_telefoni/Nokia-105-Black--1GF019CPA2L03-",
    "title": "NOKIA 105, 1.8\", 1000MAH, BLACK",
    "warranty": "24",
    "regular_price": "1.599",
    "happy_price": "",
    "description": [
        "Мобилен телефон\nЕкран:1.8\" TFT LCD\nРезолуција: 160x120pixels, 111ppi, 4:3\nОтпорност на прав и вода: IP52\n2G мрежа\nРадио\nИзлез за слушалки\nПолнење: Micro-USB\nКапацитет на батеријата: 1000mAh"
    ],
    "category": "mobilni_telefoni"
}
"""

In [12]:
prompt_template = """
Extract the features of the provided product and represent them as separate key-value pairs in a JSON format. Ensure that each feature is listed individually without any lists or dictionaries within the features. If the feature includes a specific attribute (e.g., size, capacity, type, port), include that as well. Make sure you include the url, title, price, happy_price, and warranty as well. Translate everything to English. Return the extracted features as a JSON object.

Input:
{
    "url": "https://www.neptun.mk/categories/mobilni_telefoni/Apple-iPhone-14-128GB-Yellow",
    "title": "APPLE IPHONE 14 128GB YELLOW",
    "warranty": "12",
    "regular_price": "61.990",
    "happy_price": "47.990",
    "description": [
        "Мобилен телефон\niOS 16.0\nIP68 водоотпорен ( до 6м за 30 мин)\nApple Pay систем, Поддршка за ултра широк опсег\nЕкран:6.1\" Super Retina XDR OLED\nРезолуција:1170 x 2532p\nChipset:Apple A15 Bionic (5 nm)\nCPU:Hexa-core (2x3.23 GHz Avalanche + 4x1.82 GHz Blizzard)\nGPU:Apple GPU (5-core graphics)\nИнтерна меморија:128GB\nПримарна камера:\n12 MP, f/1.5, 26mm (wide), 1.9µm, dual pixel PDAF, sensor-shift OIS\n12 MP, f/2.4, 13mm, 120? (ultrawide)\nСекундарна камера:\n12 MP, f/1.9, 23mm (wide), 1/3.6\", PDAF\nSL 3D, (depth/biometrics sensor)\nWi-Fi 802.11 a/b/g/n/ac/6, dual-band, hotspot\nBluetooth 5.3, A2DP, LE\nGPS A-GPS, GLONASS, GALILEO, BDS, QZSS\nUSB Lightning, USB 2.0\nFast charging, 50% in 30 min (advertised)\nUSB Power Delivery 2.0\nMagSafe wireless charging 15W\nQi magnetic fast wireless charging 7.5W\nКапацитет на батерија: 3279mAh"
    ],
    "category": "mobilni_telefoni"
}
Output:
{
  "url": "https://www.neptun.mk/categories/mobilni_telefoni/Apple-iPhone-14-128GB-Yellow",
  "title": "APPLE IPHONE 14 128GB YELLOW",
  "warranty": "12",
  "regular_price": "61.990",
  "happy_price": "47.990",
  "features": {
    "Product Type": "Mobile Phone",
    "Operating System": "iOS 16.0",
    "Water Resistance": "IP68 (up to 6m for 30 mins)",
    "Payment System": "Apple Pay, Ultra-Wideband (UWB) support",
    "Display Size": "6.1\" Super Retina XDR OLED",
    "Resolution": "1170 x 2532 pixels",
    "Chipset": "Apple A15 Bionic (5 nm)",
    "CPU": "Hexa-core (2x3.23 GHz Avalanche + 4x1.82 GHz Blizzard)",
    "GPU": "Apple GPU (5-core graphics)",
    "Internal Memory": "128GB",
    "Primary Camera 1": "12 MP, f/1.5, 26mm (wide), 1.9µm, dual pixel PDAF, sensor-shift OIS",
    "Primary Ultrawide Camera": "12 MP, f/2.4, 13mm, 120° (ultrawide)",
    "Secondary Camera": "12 MP, f/1.9, 23mm (wide), 1/3.6\", PDAF",
    "Depth Sensor": "SL 3D, (depth/biometrics sensor)",
    "Wi-Fi": "Wi-Fi 802.11 a/b/g/n/ac/6, dual-band, hotspot",
    "Bluetooth": "Bluetooth 5.3, A2DP, LE",
    "GPS": "A-GPS, GLONASS, GALILEO, BDS, QZSS",
    "USB": "Lightning, USB 2.0",
    "Fast Charging": "50% in 30 mins (advertised)",
    "USB Power Delivery": "USB Power Delivery 2.0",
    "Wireless Charging 1": "MagSafe wireless charging 15W",
    "Wireless Charging 2": "Qi magnetic fast wireless charging 7.5W",
    "Battery Capacity": "3279mAh"
  }
}

Input:
{
    "url": "https://www.neptun.mk/categories/mobilni_telefoni/SAMSUNG-Galaxy-A23-5G-4-64GB--SM-A236BZWUEUC--White",
    "title": "SAMSUNG GALAXY A23 5G, 6.5\", 4/64GB, 5000 MAH (SM-A236BZWUEUC) WHITE",
    "warranty": "24",
    "regular_price": "19.399",
    "happy_price": "9.999",
    "description": [
        "Мобилен телефон\nAndroid 11\nЕкран:6.5\" (720 x 1600) PLS TFT\nCPU:Octa-core (2x1.6 GHz & 6x1.6 GHz)\nИнтерна меморија:\nROM Меморија:64GB\nRAM Меморија:4GB\nПримарна камера:\n48 MP, f/1.8, (wide), AF\n2 MP, f/2.4, (depth)\nСекундарна камера:\n5 MP, f/2.2\nWi-Fi 802.11 a/b/g/n/ac, dual-band, Wi-Fi Direct, hotspot\nBluetooth 5.0, A2DP, LE\nGPS with A-GPS\nmicroUSB 2.0\nБатерија:Li-Po 5000 mAh"
    ],
    "category": "mobilni_telefoni"
}
Output:
{
  "url": "https://www.neptun.mk/categories/mobilni_telefoni/SAMSUNG-Galaxy-A23-5G-4-64GB--SM-A236BZWUEUC--White",
  "title": "SAMSUNG GALAXY A23 5G, 6.5\", 4/64GB, 5000 MAH (SM-A236BZWUEUC) WHITE",
  "warranty": "24",
  "regular_price": "19.399",
  "happy_price": "9.999",
  "features": {
    "Product Type": "Mobile Phone",
    "Operating System": "Android 11",
    "Display Size": "6.5\"",
    "Resolution": "720 x 1600 pixels, PLS TFT",
    "CPU": "Octa-core (2x1.6 GHz & 6x1.6 GHz)",
    "Internal Memory": "64GB",
    "RAM": "4GB",
    "Primary Camera 1": "48 MP, f/1.8, (wide), AF",
    "Primary Depth Camera": "2 MP, f/2.4, (depth)",
    "Secondary Camera": "5 MP, f/2.2",
    "Wi-Fi": "Wi-Fi 802.11 a/b/g/n/ac, dual-band, Wi-Fi Direct, hotspot",
    "Bluetooth": "Bluetooth 5.0, A2DP, LE",
    "GPS": "A-GPS",
    "USB": "microUSB 2.0",
    "Battery": "Li-Po 5000 mAh"
  }
}


Input:
{
    "url": "https://www.neptun.mk/categories/mobilni_telefoni/Xiaomi-Redmi-Note-12-Pro-8-256GB-Graphite-Gray",
    "title": "XIAOMI REDMI NOTE 12 PRO 6.67\", 8/256GB, 108+8+5+2/16MP, 5000 MAH, GRAPHITE GRAY",
    "warranty": "24",
    "regular_price": "20.999",
    "happy_price": "14.999",
    "description": [
        "Мобилен телефон\nAndroid 12, MIUI 13\nЕкран:6.67\" (1080 x 2400) AMOLED, 120Hz\nCorning Gorilla Glass 5\nОтпорност на вода:IP53\nQualcomm SM7150 Snapdragon 732G (8 nm)\nOcta-core (2x2.3 GHz Kryo 470 Gold & 6x1.8 GHz Kryo 470 Silver)\nAdreno 618\nИнтерна меморија:\nROM Меморија:256GB\nRAM Меморија:8GB\nПримарна камера:\n108 MP, f/1.9, 1/1.52\", 0.7µm, PDAF, (wide)\n8 MP, f/1.9, 119?, 1/4\", 1.12µm, (ultrawide)\n5 MP, f/2.4, (macro)\n2 MP, f/2.4, (depth)\nСекундарна камера:\n16 MP, 1/3.06\", 1.0µm, (wide)\nWi-Fi 802.11 a/b/g/n/ac, dual-band\nBluetooth 5.1, A2DP, LE\nGPS (L1), GLONASS (G1), BDS (B1I+B1c), GALILEO (E1), QZSS (L1)\nUSB Type-C 2.0, OTG\nFast charging 67W wired\nБатерија:Li-Po 5000 mAh"
    ],
    "category": "mobilni_telefoni"
}
Output:
{
  "url": "https://www.neptun.mk/categories/mobilni_telefoni/Xiaomi-Redmi-Note-12-Pro-8-256GB-Graphite-Gray",
  "title": "XIAOMI REDMI NOTE 12 PRO 6.67\", 8/256GB, 108+8+5+2/16MP, 5000 MAH, GRAPHITE GRAY",
  "warranty": "24",
  "regular_price": "20.999",
  "happy_price": "14.999",
  "features": {
    "Product Type": "Mobile Phone",
    "Operating System": "Android 12, MIUI 13",
    "Display Size": "6.67\"",
    "Resolution": "1080 x 2400 pixels, AMOLED, 120Hz",
    "Display Protection": "Corning Gorilla Glass 5",
    "Water Resistance": "IP53",
    "Chipset": "Qualcomm SM7150 Snapdragon 732G (8 nm)",
    "CPU": "Octa-core (2x2.3 GHz Kryo 470 Gold & 6x1.8 GHz Kryo 470 Silver)",
    "GPU": "Adreno 618",
    "Internal Memory": "256GB",
    "RAM": "8GB",
    "Primary Camera 1": "108 MP, f/1.9, 1/1.52\", 0.7µm, PDAF, (wide)",
    "Primary Ultrawide Camera": "8 MP, f/1.9, 119°, 1/4\", 1.12µm, (ultrawide)",
    "Primary Macro Camera": "5 MP, f/2.4, (macro)",
    "Primary Depth Camera": "2 MP, f/2.4, (depth)",
    "Secondary Camera": "16 MP, 1/3.06\", 1.0µm, (wide)",
    "Wi-Fi": "Wi-Fi 802.11 a/b/g/n/ac, dual-band",
    "Bluetooth": "Bluetooth 5.1, A2DP, LE",
    "GPS": "GPS (L1), GLONASS (G1), BDS (B1I+B1c), GALILEO (E1), QZSS (L1)",
    "USB": "Type-C 2.0, OTG",
    "Fast Charging": "67W wired",
    "Battery": "Li-Po 5000 mAh"
  }
}

Provided product:
{{text}}
"""
prompt = PromptTemplate(template=prompt_template, input_variables=["text"], template_format="jinja2")
chain = LLMChain(llm=llm, prompt=prompt)

In [13]:
structured_data = chain.invoke({"text": text})

print(structured_data['text'])

Output:
{
  "url": "https://www.neptun.mk/categories/mobilni_telefoni/Nokia-105-Black--1GF019CPA2L03-",
  "title": "NOKIA 105, 1.8", 1000MAH, BLACK",
  "warranty": "24",
  "regular_price": "1.599",
  "features": {
    "Product Type": "Mobile Phone",
    "Display Size": "1.8\"",
    "Display Type": "TFT LCD",
    "Resolution": "160x120 pixels, 111ppi, 4:3",
    "Water and Dust Resistance": "IP52",
    "Network": "2G",
    "Radio": "Yes",
    "Headphone Jack": "Yes",
    "Charging Port": "Micro-USB",
    "Battery Capacity": "1000mAh"
  }
}


In [15]:
# Directory containing JSON files
input_directory = r"C:\Users\tomce\OneDrive - UKIM, FINKI\Desktop\Fakultet 3ta Godina\2 Sesti Semestar\0 DATA SCIENCE SEMINARSKA\1 Starting Over\products_with_categories\mobilni_telefoni"
# Directory to save processed JSON files
output_directory = r"C:\Users\tomce\OneDrive - UKIM, FINKI\Desktop\Fakultet 3ta Godina\2 Sesti Semestar\0 DATA SCIENCE SEMINARSKA\1 Starting Over\products_categories_fixed\mobile_phones\mobile_phones_gemini"

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

In [16]:
# Loop through all files in the input directory
for filename in os.listdir(input_directory):
    if filename.endswith('.json'):
        input_filepath = os.path.join(input_directory, filename)
        
        # Open and read each JSON file
        with open(input_filepath, 'r', encoding='utf-8') as file:
            data = json.load(file)
            text = json.dumps(data)
            
            # Process the JSON data using the LangChain model
            structured_data = chain.invoke({"text": text})
            if 'text' in structured_data:
                try:
                    structured_data = json.loads(structured_data['text'])
                except JSONDecodeError:
                    continue
            
            # Determine the output file path
            output_filename = f"processed_{filename}"
            output_filepath = os.path.join(output_directory, output_filename)
            
            # Write the processed data to the output file
            with open(output_filepath, 'w', encoding='utf-8') as outfile:
                json.dump(structured_data, outfile, indent=4)

print(f"Processed JSON files are saved in {output_directory}")

Processed JSON files are saved in C:\Users\tomce\OneDrive - UKIM, FINKI\Desktop\Fakultet 3ta Godina\2 Sesti Semestar\0 DATA SCIENCE SEMINARSKA\1 Starting Over\products_categories_fixed\mobile_phones\mobile_phones_gemini


#### Vagja outputs so features{} emtpy

#### For Single File:

In [20]:
# Specify the input and output file paths
input_file = r"C:\Users\tomce\OneDrive - UKIM, FINKI\Desktop\Fakultet 3ta Godina\2 Sesti Semestar\0 DATA SCIENCE SEMINARSKA\1 Starting Over\products_categories_fixed\mobile_phones\mobile_phones_gemini\processed_Apple-iPhone-15-Pro-Max-256GB-Natural-Titanium.json"
output_file = r"C:\Users\tomce\OneDrive - UKIM, FINKI\Desktop\Fakultet 3ta Godina\2 Sesti Semestar\0 DATA SCIENCE SEMINARSKA\1 Starting Over\products_categories_fixed\mobile_phones\processed_Apple-iPhone-15-Pro-Max-256GB-Natural-Titanium.json"

# Open and read the input JSON file
with open(input_file, 'r', encoding='utf-8') as file:
    data = json.load(file)
    text = json.dumps(data)

    # Process the JSON data using the LangChain model
    structured_data = chain.invoke({"text": text})
    
    # Write the processed data to the output file
    with open(output_file, 'w', encoding='utf-8') as outfile:
        json.dump(structured_data, outfile, indent=4)

print(f"Processed JSON file saved to {output_file}")


Processed JSON file saved to C:\Users\tomce\OneDrive - UKIM, FINKI\Desktop\Fakultet 3ta Godina\2 Sesti Semestar\0 DATA SCIENCE SEMINARSKA\1 Starting Over\products_categories_fixed\mobile_phones\processed_Apple-iPhone-15-Pro-Max-256GB-Natural-Titanium.json


In [21]:
print(structured_data['text'])

{
  "url": "https://www.neptun.mk/categories/mobilni_telefoni/Apple-iPhone-15-Pro-Max-256GB-Natural-Titanium",
  "title": "IPHONE 15 PRO MAX 256GB NATURAL TITANIUM",
  "warranty": "12",
  "regular_price": "97.490",
  "happy_price": "87.990",
  "features": {}
}
